## Deploying to Vercel

In this notebook, we will deploy our Next.js application to Vercel using the Vercel REST API. We will follow these steps:

1. **Create a Deployment:** We will create a deployment by making an HTTP POST request to the `/v12/now/deployments` endpoint of the Vercel API. The body of the request will include all the files we want to deploy. We will need to generate a SHA for each file and upload it using the Vercel API.

2. **Add Environment Variables:** After creating the deployment, we will add our environment variables (like the Stripe API key) to the deployment. We will do this by making an HTTP PATCH request to the `/v5/projects/{projectId}/env` endpoint of the Vercel API. The body of the request will include the names and values of the environment variables.

3. **Assign a Domain:** Once the deployment is created and the environment variables are set, we will assign a domain to our deployment. We will do this by making an HTTP POST request to the `/v4/domains/{domain}/records` endpoint of the Vercel API. The body of the request will include the name of the domain we want to assign.

4. **Check the Deployment Status:** After assigning the domain, we will check the status of our deployment by making an HTTP GET request to the `/v12/now/deployments/{deploymentId}` endpoint of the Vercel API. The response will include information about the deployment, including its status.

Let's start by importing the necessary Python libraries.

In [ ]:
import os
import requests
import json
from getpass import getpass

## Step 1: Create a Deployment

We will start by creating a deployment. To do this, we will make an HTTP POST request to the `/v12/now/deployments` endpoint of the Vercel API. The body of the request will include all the files we want to deploy.

First, let's define the URL of the endpoint and the headers of our request. We will need to provide our Vercel API key in the `Authorization` header.

In [ ]:
VERCEL_API_KEY = os.getenv('VERCEL_API_KEY')

url = 'https://api.vercel.com/v12/now/deployments'
headers = {
    'Authorization': f'Bearer {VERCEL_API_KEY}',
    'Content-Type': 'application/json'
}

Next, we need to prepare the body of our request. The body should include all the files we want to deploy. Each file should be represented as a dictionary with two keys: `file` and `data`. The `file` key should have the path of the file as its value, and the `data` key should have the content of the file as its value.

Let's define a function that takes a list of file paths and returns the body of our request.

In [ ]:
def prepare_body(file_paths):
    files = []
    for file_path in file_paths:
        with open(file_path, 'r') as file:
            data = file.read()
        files.append({'file': file_path, 'data': data})
    return {'files': files}

Now, let's use this function to prepare the body of our request. We will deploy the `README.md` file and the `human_ai_conflict.png` image file. We need to specify the paths of these files relative to the root directory of our project.

In [ ]:
file_paths = ['README.md', 'human_ai_conflict.png']
body = prepare_body(file_paths)

## Step 2: Add Environment Variables

After creating the deployment, we need to add our environment variables to the deployment. We can do this by making an HTTP PATCH request to the `/v5/projects/{projectId}/env` endpoint of the Vercel API. The body of the request should include the names and values of the environment variables.

First, let's define the URL of the endpoint and the headers of our request. We will need to provide our Vercel API key in the `Authorization` header.

In [ ]:
project_id = 'your-project-id'  # replace with your project ID

url = f'https://api.vercel.com/v5/projects/{project_id}/env'
headers = {
    'Authorization': f'Bearer {VERCEL_API_KEY}',
    'Content-Type': 'application/json'
}

Next, we need to prepare the body of our request. The body should include the names and values of the environment variables we want to add. Each environment variable should be represented as a dictionary with three keys: `key`, `value`, and `target`. The `key` key should have the name of the variable as its value, the `value` key should have the value of the variable as its value, and the `target` key should have the target of the variable (either `production`, `preview`, or `development`) as its value.

Let's define a function that takes a dictionary of environment variables and returns the body of our request.

In [ ]:
def prepare_body(env_vars):
    envs = []
    for key, value in env_vars.items():
        envs.append({'key': key, 'value': value, 'target': 'production'})
    return {'envs': envs}

Now, let's use this function to prepare the body of our request. We will add the `STRIPE_API_KEY` environment variable to our deployment. We need to specify the value of this variable.

In [ ]:
env_vars = {'STRIPE_API_KEY': 'your-stripe-api-key'}  # replace with your Stripe API key
body = prepare_body(env_vars)

Finally, we can make the HTTP PATCH request to add our environment variables to the deployment. We will use the `requests` library to make the request. We need to provide the URL, headers, and body of our request as arguments to the `requests.patch` function.

In [ ]:
import requests

response = requests.patch(url, headers=headers, json=body)
response.json()

## Step 3: Create README.md

Let's create a README.md file with a Donation button that links to the Donation product in Stripe. We will use Python's built-in file handling functions to create the file and write the markdown content to it.

In [ ]:
readme_content = """
# Stripe Payment Project

This project was entirely developed using ChatGPT. This is made possible because of the Noteable plugin being able to execute Python code.

The sample project we are working on is various implementations of the Stripe donation button.

[![Donate Now](https://img.shields.io/badge/Donate-Now-green.svg)](https://stripe.com/checkout)
"""

with open('README.md', 'w') as file:
    file.write(readme_content)

## Step 4: Update Payment Link

We need to update the payment link in the README.md file to match the 5.0 price on the donation product. We also need to allow the user to adjust the quantity between 1 and 100. We can do this by accessing the Stripe API using Python code.

In [ ]:
import stripe

stripe.api_key = os.environ['STRIPE_API_KEY']

# Retrieve the product
product = stripe.Product.list(limit=1)['data'][0]

# Retrieve the prices
prices = stripe.Price.list(product=product['id'], limit=3)['data']

# Find the price that matches 5.0
price_5 = next((price for price in prices if price['unit_amount'] / 100 == 5.0), None)

if price_5:
    # Update the README.md content
    readme_content = readme_content.replace('https://stripe.com/checkout', 'https://checkout.stripe.com/pay/' + price_5['id'])

    # Write the updated content back to the file
    with open('README.md', 'w') as file:
        file.write(readme_content)

## Step 5: Update README.md

We need to update the README.md file to reflect the new project description. We can do this by modifying the content of the README.md file.

In [ ]:
readme_content = """
# Stripe Payment Project

This project was entirely developed using ChatGPT and the Noteable plugin. It demonstrates the following implementations:

- A Stripe donation button, including the creation of the payment link via the Stripe API.
- Deploying code to GitHub using the GitHub API.
- Creation of a GitHub README.md that includes a generated image.
- A NodeJS Stripe deployment to Vercel hosting via the Vercel API.

[![Donate Now](https://img.shields.io/badge/Donate-Now-green.svg)](https://checkout.stripe.com/pay/" + price_5['id'] + ")
"""

with open('README.md', 'w') as file:
    file.write(readme_content)

## Step 6: Update README.md with New Payment Link and QR Code

We need to update the README.md file to include the new payment link and the QR code. We can do this by modifying the content of the README.md file.

In [ ]:
readme_content = """
# Stripe Payment Project

This project was entirely developed using ChatGPT and the Noteable plugin. It demonstrates the following implementations:

- A Stripe donation button, including the creation of the payment link via the Stripe API.
- Deploying code to GitHub using the GitHub API.
- Creation of a GitHub README.md that includes a generated image.
- A NodeJS Stripe deployment to Vercel hosting via the Vercel API.

[![Donate Now](https://img.shields.io/badge/Donate-Now-green.svg)](https://buy.stripe.com/00g14peASeEd7xCcMM)

![QR Code](qr_00g14peASeEd7xCcMM.png)
"""

with open('README.md', 'w') as file:
    file.write(readme_content)

In [ ]:
# Push the README.md and the QR code image to the GitHub repository

commit_message = 'Updated README.md and added QR code image'
files_to_commit = ['README.md', 'qr_00g14peASeEd7xCcMM.png']

for file in files_to_commit:
    with open(file, 'rb') as f:
        content = f.read()

    # Create a blob for the file
    blob_sha = repo.create_blob(content, 'base64')

    # Add the blob to the tree
    tree.append(github.InputGitTreeElement(path=file, mode='100644', type='blob', sha=blob_sha))

# Create a new tree
tree = repo.create_git_tree(tree)

# Create a new commit
parent = repo.get_git_commit(master_ref.object.sha)
commit = repo.create_git_commit(commit_message, tree, [parent])

# Update the master branch to point to the new commit
master_ref.edit(commit.sha)

In [ ]:
from github import Github

# First create a Github instance:

# using an access token
g = Github(os.getenv('GITHUB_TOKEN'))

# Then get your repository
repo = g.get_user().get_repo('stripe-payment-project')

# Push the README.md and the QR code image to the GitHub repository

commit_message = 'Updated README.md and added QR code image'
files_to_commit = ['README.md', 'qr_00g14peASeEd7xCcMM.png']

tree = []

for file in files_to_commit:
    with open(file, 'rb') as f:
        content = f.read()

    # Create a blob for the file
    blob_sha = repo.create_blob(content, 'base64')

    # Add the blob to the tree
    tree.append(github.InputGitTreeElement(path=file, mode='100644', type='blob', sha=blob_sha))

# Create a new tree
tree = repo.create_git_tree(tree)

# Create a new commit
parent = repo.get_git_commit(master_ref.object.sha)
commit = repo.create_git_commit(commit_message, tree, [parent])

# Update the master branch to point to the new commit
master_ref.edit(commit.sha)

In [ ]:
!pip install PyGithub

## Step 7: Update README.md with TODO and Quote

We need to update the README.md file to include the TODO for Vercel deployment and a quote. We can do this by modifying the content of the README.md file.

In [ ]:
readme_content = """
# Stripe Payment Project

This project was entirely developed using ChatGPT and the Noteable plugin. It demonstrates the following implementations:

- A Stripe donation button, including the creation of the payment link via the Stripe API.
- Deploying code to GitHub using the GitHub API.
- Creation of a GitHub README.md that includes a generated image.
- A NodeJS Stripe deployment to Vercel hosting via the Vercel API.

TODO: Vercel deployment

"Imagination is not what we think, it is how we think" -- Unknown

[![Donate Now](https://img.shields.io/badge/Donate-Now-green.svg)](https://buy.stripe.com/00g14peASeEd7xCcMM)

![QR Code](qr_00g14peASeEd7xCcMM.png)
"""

with open('README.md', 'w') as file:
    file.write(readme_content)

## Step 8: Create a Subfolder on GitHub and Push Notebooks

We will create a subfolder on GitHub called 'notebooks' and write a Python script to push all local notebooks with extension .ipynb to this subfolder.

In [ ]:
import os
import glob

# Get all .ipynb files in the current directory
notebooks = glob.glob('*.ipynb')

# Create a new tree element for the 'notebooks' directory
tree.append(github.InputGitTreeElement(path='notebooks', mode='040000', type='tree'))

# For each notebook, create a blob and add it to the tree
for notebook in notebooks:
    with open(notebook, 'rb') as f:
        content = f.read()

    # Create a blob for the notebook
    blob_sha = repo.create_blob(content, 'base64')

    # Add the blob to the tree
    tree.append(github.InputGitTreeElement(path='notebooks/' + notebook, mode='100644', type='blob', sha=blob_sha))

# Create a new tree
tree = repo.create_git_tree(tree)

# Create a new commit
parent = repo.get_git_commit(master_ref.object.sha)
commit = repo.create_git_commit('Added notebooks', tree, [parent])

# Update the master branch to point to the new commit
master_ref.edit(commit.sha)

In [ ]:
import os
import glob

# Get all .ipynb files in the current directory
notebooks = glob.glob('*.ipynb')

# Get the current tree of the repository
tree = repo.get_git_tree(master_ref.object.sha)

# Create a new tree element for the 'notebooks' directory
tree = tree.tree + [github.InputGitTreeElement(path='notebooks', mode='040000', type='tree')]

# For each notebook, create a blob and add it to the tree
for notebook in notebooks:
    with open(notebook, 'rb') as f:
        content = f.read()

    # Create a blob for the notebook
    blob_sha = repo.create_blob(content, 'base64')

    # Add the blob to the tree
    tree = tree + [github.InputGitTreeElement(path='notebooks/' + notebook, mode='100644', type='blob', sha=blob_sha)]

# Create a new tree
tree = repo.create_git_tree(tree)

# Create a new commit
parent = repo.get_git_commit(master_ref.object.sha)
commit = repo.create_git_commit('Added notebooks', tree, [parent])

# Update the master branch to point to the new commit
master_ref.edit(commit.sha)

In [ ]:
from github import Github

# First create a Github instance:

# using an access token
g = Github(os.getenv('GITHUB_TOKEN'))

# Then get your repository
repo = g.get_user().get_repo('stripe-payment-project')

# Get the master branch reference
master_ref = repo.get_git_ref('heads/master')

import os
import glob

# Get all .ipynb files in the current directory
notebooks = glob.glob('*.ipynb')

# Get the current tree of the repository
tree = repo.get_git_tree(master_ref.object.sha)

# Create a new tree element for the 'notebooks' directory
tree = tree.tree + [github.InputGitTreeElement(path='notebooks', mode='040000', type='tree')]

# For each notebook, create a blob and add it to the tree
for notebook in notebooks:
    with open(notebook, 'rb') as f:
        content = f.read()

    # Create a blob for the notebook
    blob_sha = repo.create_blob(content, 'base64')

    # Add the blob to the tree
    tree = tree + [github.InputGitTreeElement(path='notebooks/' + notebook, mode='100644', type='blob', sha=blob_sha)]

# Create a new tree
tree = repo.create_git_tree(tree)

# Create a new commit
parent = repo.get_git_commit(master_ref.object.sha)
commit = repo.create_git_commit('Added notebooks', tree, [parent])

# Update the master branch to point to the new commit
master_ref.edit(commit.sha)

In [ ]:
from github import Github

# First create a Github instance:

# using an access token
g = Github(os.getenv('GITHUB_TOKEN'))

# Then get your repository
repo = g.get_user().get_repo('stripe-payment')

# Get the master branch reference
master_ref = repo.get_git_ref('heads/master')

import os
import glob

# Get all .ipynb files in the current directory
notebooks = glob.glob('*.ipynb')

# Get the current tree of the repository
tree = repo.get_git_tree(master_ref.object.sha)

# Create a new tree element for the 'notebooks' directory
tree = tree.tree + [github.InputGitTreeElement(path='notebooks', mode='040000', type='tree')]

# For each notebook, create a blob and add it to the tree
for notebook in notebooks:
    with open(notebook, 'rb') as f:
        content = f.read()

    # Create a blob for the notebook
    blob_sha = repo.create_blob(content, 'base64')

    # Add the blob to the tree
    tree = tree + [github.InputGitTreeElement(path='notebooks/' + notebook, mode='100644', type='blob', sha=blob_sha)]

# Create a new tree
tree = repo.create_git_tree(tree)

# Create a new commit
parent = repo.get_git_commit(master_ref.object.sha)
commit = repo.create_git_commit('Added notebooks', tree, [parent])

# Update the master branch to point to the new commit
master_ref.edit(commit.sha)

In [ ]:
from github import Github

# First create a Github instance:

# using an access token
g = Github(os.getenv('GITHUB_TOKEN'))

# Then get your repository
repo = g.get_user().get_repo('stripe-payment')

# Get the main branch reference
main_ref = repo.get_git_ref('heads/main')

import os
import glob

# Get all .ipynb files in the current directory
notebooks = glob.glob('*.ipynb')

# Get the current tree of the repository
tree = repo.get_git_tree(main_ref.object.sha)

# Create a new tree element for the 'notebooks' directory
tree = tree.tree + [github.InputGitTreeElement(path='notebooks', mode='040000', type='tree')]

# For each notebook, create a blob and add it to the tree
for notebook in notebooks:
    with open(notebook, 'rb') as f:
        content = f.read()

    # Create a blob for the notebook
    blob_sha = repo.create_blob(content, 'base64')

    # Add the blob to the tree
    tree = tree + [github.InputGitTreeElement(path='notebooks/' + notebook, mode='100644', type='blob', sha=blob_sha)]

# Create a new tree
tree = repo.create_git_tree(tree)

# Create a new commit
parent = repo.get_git_commit(main_ref.object.sha)
commit = repo.create_git_commit('Added notebooks', tree, [parent])

# Update the main branch to point to the new commit
main_ref.edit(commit.sha)

In [ ]:
from github import Github

# First create a Github instance:

# using an access token
g = Github(os.getenv('GITHUB_TOKEN'))

# Then get your repository
repo = g.get_user().get_repo('stripe-payment')

# Get the main branch reference
main_ref = repo.get_git_ref('heads/main')

import os
import glob

# Get all .ipynb files in the current directory
notebooks = glob.glob('*.ipynb')

# Get the current tree of the repository
tree = repo.get_git_tree(main_ref.object.sha)

# Create a new tree element for the 'notebooks' directory
tree = tree.tree + [Github.InputGitTreeElement(path='notebooks', mode='040000', type='tree')]

# For each notebook, create a blob and add it to the tree
for notebook in notebooks:
    with open(notebook, 'rb') as f:
        content = f.read()

    # Create a blob for the notebook
    blob_sha = repo.create_blob(content, 'base64')

    # Add the blob to the tree
    tree = tree + [Github.InputGitTreeElement(path='notebooks/' + notebook, mode='100644', type='blob', sha=blob_sha)]

# Create a new tree
tree = repo.create_git_tree(tree)

# Create a new commit
parent = repo.get_git_commit(main_ref.object.sha)
commit = repo.create_git_commit('Added notebooks', tree, [parent])

# Update the main branch to point to the new commit
main_ref.edit(commit.sha)

In [ ]:
from github import Github, InputGitTreeElement

# First create a Github instance:

# using an access token
g = Github(os.getenv('GITHUB_TOKEN'))

# Then get your repository
repo = g.get_user().get_repo('stripe-payment')

# Get the main branch reference
main_ref = repo.get_git_ref('heads/main')

import os
import glob

# Get all .ipynb files in the current directory
notebooks = glob.glob('*.ipynb')

# Get the current tree of the repository
tree = repo.get_git_tree(main_ref.object.sha)

# Create a new tree element for the 'notebooks' directory
tree = tree.tree + [InputGitTreeElement(path='notebooks', mode='040000', type='tree')]

# For each notebook, create a blob and add it to the tree
for notebook in notebooks:
    with open(notebook, 'rb') as f:
        content = f.read()

    # Create a blob for the notebook
    blob_sha = repo.create_blob(content, 'base64')

    # Add the blob to the tree
    tree = tree + [InputGitTreeElement(path='notebooks/' + notebook, mode='100644', type='blob', sha=blob_sha)]

# Create a new tree
tree = repo.create_git_tree(tree)

# Create a new commit
parent = repo.get_git_commit(main_ref.object.sha)
commit = repo.create_git_commit('Added notebooks', tree, [parent])

# Update the main branch to point to the new commit
main_ref.edit(commit.sha)

In [ ]:
from github import Github

# First create a Github instance:

# using an access token
g = Github(os.getenv('GITHUB_TOKEN'))

# Then get your repository
repo = g.get_user().get_repo('stripe-payment')

# Get the main branch reference
main_ref = repo.get_git_ref('heads/main')

import os
import glob

# Get all .ipynb files in the current directory
notebooks = glob.glob('*.ipynb')

# Get the current tree of the repository
tree = repo.get_git_tree(main_ref.object.sha)

# Create a new tree element for the 'notebooks' directory
tree = tree.tree + [Github.InputGitTreeElement(path='notebooks', mode='040000', type='tree')]

# For each notebook, create a blob and add it to the tree
for notebook in notebooks:
    with open(notebook, 'rb') as f:
        content = f.read()

    # Create a blob for the notebook
    blob_sha = repo.create_blob(content, 'base64')

    # Add the blob to the tree
    tree = tree + [Github.InputGitTreeElement(path='notebooks/' + notebook, mode='100644', type='blob', sha=blob_sha)]

# Create a new tree
tree = repo.create_git_tree(tree)

# Create a new commit
parent = repo.get_git_commit(main_ref.object.sha)
commit = repo.create_git_commit('Added notebooks', tree, [parent])

# Update the main branch to point to the new commit
main_ref.edit(commit.sha)

In [ ]:
from github import Github, InputGitTreeElement

# First create a Github instance:

# using an access token
g = Github(os.getenv('GITHUB_TOKEN'))

# Then get your repository
repo = g.get_user().get_repo('stripe-payment')

# Get the main branch reference
main_ref = repo.get_git_ref('heads/main')

import os
import glob

# Get all .ipynb files in the current directory
notebooks = glob.glob('*.ipynb')

# Get the current tree of the repository
tree = repo.get_git_tree(main_ref.object.sha)

# Create a new tree element for the 'notebooks' directory
tree = tree.tree + [InputGitTreeElement(path='notebooks', mode='040000', type='tree')]

# For each notebook, create a blob and add it to the tree
for notebook in notebooks:
    with open(notebook, 'rb') as f:
        content = f.read()

    # Create a blob for the notebook
    blob_sha = repo.create_blob(content, 'base64')

    # Add the blob to the tree
    tree = tree + [InputGitTreeElement(path='notebooks/' + notebook, mode='100644', type='blob', sha=blob_sha)]

# Create a new tree
tree = repo.create_git_tree(tree)

# Create a new commit
parent = repo.get_git_commit(main_ref.object.sha)
commit = repo.create_git_commit('Added notebooks', tree, [parent])

# Update the main branch to point to the new commit
main_ref.edit(commit.sha)

In [ ]:
from github import Github, InputGitTreeElement

# First create a Github instance:

# using an access token
g = Github(os.getenv('GITHUB_TOKEN'))

# Then get your repository
repo = g.get_user().get_repo('stripe-payment')

# Get the main branch reference
main_ref = repo.get_git_ref('heads/main')

import os
import glob

# Get all .ipynb files in the current directory
notebooks = glob.glob('*.ipynb')

# Get the current tree of the repository
tree = repo.get_git_tree(main_ref.object.sha)

# Create a new tree element for the 'notebooks' directory
tree = tree.tree + [InputGitTreeElement(path='notebooks', mode='040000', type='tree')]

# For each notebook, create a blob and add it to the tree
for notebook in notebooks:
    with open(notebook, 'rb') as f:
        content = f.read()

    # Create a blob for the notebook
    blob_sha = repo.create_git_blob(content, 'base64')

    # Add the blob to the tree
    tree = tree + [InputGitTreeElement(path='notebooks/' + notebook, mode='100644', type='blob', sha=blob_sha)]

# Create a new tree
tree = repo.create_git_tree(tree)

# Create a new commit
parent = repo.get_git_commit(main_ref.object.sha)
commit = repo.create_git_commit('Added notebooks', tree, [parent])

# Update the main branch to point to the new commit
main_ref.edit(commit.sha)